### 说明
* 对1得到的label1:1完整数据集转换ffm
* 输入目录

path = 'data1'         # 为了方便不和前两个文件夹冲突,在同级目录创建data1,专门存放数据转换好以后用来预测的数据文件

train_name = 'data.csv'      # 1之后得到的label 1:1 数据集   应该是在data/01data/里面

test_name = 'test.csv'        # 测试集文件,要求是merge以后的文件

train_temp_name = 'train_data.csv'   # ffm转换后的临时文件

train_ffm_name = 'train_ffm.csv'      # ffm输入的训练集文件

test_ffm_name = 'test_ffm.csv'       #  ffm输入的测试集文件

In [1]:
import pandas as pd
from pandas import get_dummies
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import numpy as np
import os
import gc
import xlearn as xl

In [2]:
class FFMFormat:
    def __init__(self,vector_feat,one_hot_feat,continus_feat):
        self.field_index_ = None
        self.feature_index_ = None
        self.vector_feat=vector_feat
        self.one_hot_feat=one_hot_feat
        self.continus_feat=continus_feat
        
    def get_params(self):
        pass

    def set_params(self, **parameters):
        pass

    def fit(self, df, y=None):
        self.field_index_ = {col: i for i, col in enumerate(df.columns)}
        self.feature_index_ = dict()
        last_idx = 0
        for col in df.columns:
            if col in self.one_hot_feat:
                print(col)
                df[col]=df[col].astype('int')
                vals = np.unique(df[col])
                for val in vals:
                    if val==-1: continue
                    name = '{}_{}'.format(col, val)
                    if name not in self.feature_index_:
                        self.feature_index_[name] = last_idx
                        last_idx += 1
            elif col in self.vector_feat:
                print(col)
                vals=[]
                for data in df[col].apply(str):
                    if data!="-1":
                        for word in data.strip().split(' '):
                            vals.append(word)
                vals = np.unique(vals)
                for val in vals:
                    if val=="-1": continue
                    name = '{}_{}'.format(col, val)
                    if name not in self.feature_index_:
                        self.feature_index_[name] = last_idx
                        last_idx += 1
            self.feature_index_[col] = last_idx
            last_idx += 1
        return self

    def fit_transform(self, df, y=None):
        self.fit(df, y)
        return self.transform(df)

    def transform_row_(self, row):
        ffm = []

        for col, val in row.loc[row != 0].to_dict().items():
            if col in self.one_hot_feat:
                name = '{}_{}'.format(col, val)
                if name in self.feature_index_:
                    ffm.append('{}:{}:1'.format(self.field_index_[col], self.feature_index_[name]))
                # ffm.append('{}:{}:{}'.format(self.field_index_[col], self.feature_index_[col], 1))
            elif col in self.vector_feat:
                for word in str(val).split(' '):
                    name = '{}_{}'.format(col, word)
                    if name in self.feature_index_:
                        ffm.append('{}:{}:1'.format(self.field_index_[col], self.feature_index_[name]))
            elif col in self.continus_feat:
                if val!=-1:
                    ffm.append('{}:{}:{}'.format(self.field_index_[col], self.feature_index_[col], val))
        return ' '.join(ffm)

    def transform(self, df):
        # val=[]
        # for k,v in self.feature_index_.items():
        #     val.append(v)
        # val.sort()
        # print(val)
        # print(self.field_index_)
        # print(self.feature_index_)
        return pd.Series({idx: self.transform_row_(row) for idx, row in df.iterrows()})

In [3]:
one_hot_feature=['LBS','age','carrier','consumptionAbility','education','gender','advertiserId','campaignId', 'creativeId',
       'adCategoryId', 'productId', 'productType']
vector_feature=['interest1','interest2','interest5','kw1','kw2','topic1','topic2','os','ct','marriageStatus']
continus_feature=['creativeSize']

path = 'data1'
train_name = 'data.csv'
test_name = 'test.csv'
train_temp_name = 'train_data.csv'
train_ffm_name = 'train_ffm.csv'
test_ffm_name = 'test_ffm.csv'

In [4]:
tr = FFMFormat(vector_feature,one_hot_feature,continus_feature)
train = pd.read_csv(path+'/'+train_name)
test = pd.read_csv(path+'/'+test_name)

Y = np.array(train.pop('label'))
len_train = len(train)

train = train[one_hot_feature+vector_feature+continus_feature]
# vali = train[one_hot_feature+vector_feature+continus_feature]

tr = FFMFormat(vector_feature,one_hot_feature,continus_feature)
train_ffm = tr.fit_transform(train)
test_ffm = tr.fit_transform(test)

LBS
age
carrier
consumptionAbility
education
gender
advertiserId
campaignId
creativeId
adCategoryId
productId
productType
interest1
interest2
interest5
kw1
kw2
topic1
topic2
os
ct
marriageStatus
advertiserId
campaignId
creativeId
adCategoryId
productId
productType
LBS
age
carrier
consumptionAbility
ct
education
gender
interest1
interest2
interest5
kw1
kw2
marriageStatus
os
topic1
topic2


In [5]:
test_ffm.to_csv(path+'/'+test_ffm_name,index=False)
train_ffm.to_csv(path+'/'+train_temp_name,index=False)

with open(path+'/'+train_temp_name) as fin:
    f_train_out = open(path+'/'+train_ffm_name,'w')
    for (i,line) in enumerate(fin):
        f_train_out.write(str(Y[i])+' '+line)
    f_train_out.close()
    print('转换成功，数据位置：'+path)

del train
del test
del test_ffm
del train_ffm
gc.collect()

转换成功，数据位置：data1


5764

### 说明
* 这个时候已经得到了train_ffm和test_ffm,接下来带入模型预测
* 这部分只要数据转换完成了，是可以单独运行的，即如果是为了调参，单独运行这一部分就行

In [6]:
import pandas as pd
from pandas import get_dummies
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import numpy as np
import os
import gc
import xlearn as xl

In [7]:

path = 'data1'
test_name = 'test.csv'
train_ffm_name = 'train_ffm.csv'
test_ffm_name = 'test_ffm.csv'

In [9]:
param = {'task':'binary', 'lr':0.02, 'lambda':0.000001,'metric': 'auc','opt':'ftrl','epoch':2,'k':2,
         'alpha': 1.5, 'beta': 1, 'lambda_1': 0.0, 'lambda_2': 0.0}
# lam = [0.1,0.03,0.01,0.003,0.001,0.0003,0.0001,0.00003,0.00001,0.000003,0.000001]
# lr = [1,0.3,0.1,0.03,0.01]

ffm_model = xl.create_ffm()
ffm_model.setTrain(path+'/'+train_ffm_name)
ffm_model.setTest(path+'/'+test_ffm_name)
# ffm_model.cv(param)
# ffm_model.setValidate(path+'/ffm/'+vali_ffm_name)
ffm_model.setSigmoid()
ffm_model.fit(param,path+"/"+"model.out")
print('fit ok:'+path)
ffm_model.predict(path+"/"+"model.out",path+"/"+"output.txt")
sub = pd.DataFrame()
test_df = pd.read_csv(path+'/'+test_name)
sub['aid']=test_df['aid']
sub['uid']=test_df['uid']
sub['score'] = np.loadtxt(path+"/"+"output.txt")
sub.to_csv(path+'/submission/'+'submission.csv',index=False)

# for i in lam:
#     for j in lr:
#         param['lr'] = j
#         param['lambda'] = i
#         print(param)
#         ffm_model.cv(param)

fit ok:data1


FileNotFoundError: [Errno 2] No such file or directory: 'data1/submission/submission.csv'